# Forward Simulation of UXO Detection

In [1]:
import os
import numpy as np
from forward_freq_survey import Forward_Freq_Survey
from layered_earth import Layered_Earth
from UXO_object import UXO_Object

## Define System Parameters

In [2]:
z0 = 0.1 # height of our sensor above the ground (m)
moment = 1 # dipole moment in A*m^2
coil_spacing = 1 # spacing between Rx and Tx coils (m)
min_freq = 1e4 # minimum frequency (Hz)
max_freq = 1e5 # maximum frequency (Hz)
num_freqs = 5 # number of frequencies to use in simulation
soil_conductivity = 1 # conductivity of the soil (S/m)
uxo_conductivity = 1e6 # conductivity of the UXO (S/m)

## Define Simulated Data and x,y Grid

In [3]:
# define frequencies
frequencies = np.logspace(np.log10(min_freq), np.log10(max_freq), num_freqs)

# create uxo object
uxo = UXO_Object(np.array([0.0, 0.0, -1.0]), 0.4, 0.2, 0, 0, uxo_conductivity)

# create layered earth object
earth = Layered_Earth(soil_conductivity, uxo)

# define x,y grid
min_dist = -1
max_dist = 1
n_points = 10
x_vals = np.linspace(min_dist, max_dist, n_points)
y_vals = np.linspace(min_dist, max_dist, n_points)

# define the forward simulation object
sim = Forward_Freq_Survey(frequencies, z0, moment, coil_spacing)

## Run the simulation and save the data

In [4]:
import pickle

# get predicted data at each xy point
data_list = []
for x in x_vals:
    for y in y_vals:
        # get predicted data
        dpred = sim.predict_data(*earth.layers_at_pt((x, y)))
        # add noise to the data
        noise = (
            0.05
            * np.abs(dpred)
            * np.random.normal(size=len(dpred))
        )
        dpred_out = dpred + noise
        # store the data in a dictionary
        data_dict = {"x": x,
                     "y": y,
                     "frequencies": frequencies,
                     "hz_real": dpred_out[0::2],
                     "hz_imag": dpred_out[1::2]}
        data_list.append(data_dict)

# write data to file
os.makedirs("../forward_data", exist_ok=True)
with open("../forward_data/forward_simulation_data_test0.pkl", "wb") as f:
    pickle.dump((data_list, earth), f)

## Plot the Data
Todo: make this work with the new data structure

In [5]:
ylim = [np.min(dpred), 1.1 * np.max(dpred)]

fig = plt.figure(figsize=(10, 5))

ax1 = fig.add_axes([0.1, 0.1, 0.4, 0.85])
ax1.semilogx(frequencies, dpred[0::2], "b-o", lw=3, ms=10)
ax1.grid()
ax1.set_ylim(ylim)
ax1.set_xlabel("Frequency (Hz)")
ax1.set_ylabel("Hs/Hp (ppm)")
ax1.set_title("Real Component")

ax2 = fig.add_axes([0.55, 0.1, 0.4, 0.85])
ax2.semilogx(frequencies, dpred[1::2], "b-o", lw=3, ms=10)
ax2.set_ylim(ylim)
ax2.grid()
ax2.set_xlabel("Frequency (Hz)")
ax2.set_yticklabels("")
ax2.set_title("Imaginary Component")

plt.show()

NameError: name 'plt' is not defined